This script performs gene ranking (with SHAP) and gene selection (by p-values from random permutation) with all expressed genes

In [3]:
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
from tqdm import tqdm
from functools import reduce

Read LOO test results for polyEN applied to HLCA dataset 

In [9]:
table = pd.read_csv("results/eval/performances_HLCA_loo_polyEN.csv")

In [10]:
table_smoker = table.loc[table["group"] == "smoker",]
table_nonsmoker = table.loc[table["group"] == "nonsmoker",]

In [11]:
table_nonsmoker = table.loc[table['group'] == "nonsmoker",]
dfs = []
for i,((annLevel,ct,gene_type,use_pca),df) in enumerate(table_nonsmoker.groupby(["ann_level","cell_type","gene_type","use_pca"])):
    df["R2_mean"] = df["R2"].mean()
    dfs.append(df)
table_nonsmoker = pd.concat(dfs).sort_values(by = "R2_mean", ascending = False)

table_smoker = table.loc[table['group'] == "smoker",]
dfs = []
for i,((annLevel,ct,gene_type,use_pca),df) in enumerate(table_smoker.groupby(["ann_level","cell_type","gene_type","use_pca"])):
    df["R2_mean"] = df["R2"].mean()
    dfs.append(df)
table_smoker = pd.concat(dfs).sort_values(by = "R2_mean", ascending = False)

In [12]:
del(table_smoker["R2"],table_smoker["rep"],table_nonsmoker["R2"],table_nonsmoker["rep"])

In [13]:
table_smoker.drop_duplicates(inplace=True)
table_nonsmoker.drop_duplicates(inplace=True)

Read data

In [14]:
hlca_smoker = sc.read_h5ad("data/step1_HLCA_smoker_processed.h5ad")
hlca_nonsmoker = sc.read_h5ad("data/step1_HLCA_nonsmoker_processed.h5ad")

/jet/home/qsong/.conda/envs/lung_aging/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/jet/home/qsong/.conda/envs/lung_aging/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Keep these columns for cell meta data

In [15]:
hlca_smoker.obs = hlca_smoker.obs.loc[:,['donor_id',
                                       'age',
                                       'ann_level_1',
                                       'ann_level_2',
                                       'ann_level_3',
                                       'ann_level_4',
                                       'ann_level_5']
                                     ]
hlca_nonsmoker.obs = hlca_nonsmoker.obs.loc[:,['donor_id',
                                       'age',
                                       'ann_level_1',
                                       'ann_level_2',
                                       'ann_level_3',
                                       'ann_level_4',
                                       'ann_level_5']
                                     ]

Get highly variable genes and all types of marker genes

In [22]:
test_genes = {"fridman":["ALDH1A3", "AOPEP", "CCND1", "CD44", "CDKN1A", "CDKN1C", "CDKN2A", "CDKN2B", "CDKN2D", "CITED2",
                                "CLTB", "COL1A2","CREG1","CRYAB","CCN2","CXCL14","CYP1B1","EIF2S2","ESM1","F3","FILIP1L","FN1","GSN","GUK1","HBS1L",
                                "HPS5","HSPA2","HTATIP2","IFI16","IFNG","IGFBP1","IGFBP2","IGFBP3","IGFBP4","IGFBP5","IGFBP6","IGFBP7","IGSF3",
                                "ING1","IRF5","IRF7","ISG15","MAP1LC3B","MAP2K3","MDM2","MMP1","NDN","NME2","NRG1","OPTN","PEA15","RAB13","RAB31",
                                "RAB5B","RABGGTA","RAC1","RBL2","RGL2","RHOB","RRAS","S100A11","SERPINB2","SERPINE1","SMPD1","SMURF2","SOD1","SPARC",
                                "STAT1","TES","TFAP2A","TGFB1I1","THBS1","TNFAIP2","TNFAIP3","TP53","TSPYL5","VIM","ALDH1A1","BMI1","CCNB1","CDC25B",
                                "CKS1BP7","COL3A1","E2F4","EGR1","ID1","LAMA1","LDB2","MARCKS","CCN4"],
              "sasp2":["VEGFA", "TNFRSF12A", "TNFRSF10C", "TNFRSF10B", "TIMP2", "TIMP1", "TGFB1", "SERPINE1", "TNFRSF1A",
                                    "PLAUR", "PLAU", "MMP14", "MMP13", "MMP7", "MMP3", "MIF", "LMNA", "KITLG", "IL32", "IGFBP7", "IGFBP2",
                                     "ICAM1", "FAS", "EREG", "CXCL17", "CXCL16", "CXCL8", "CXCL1", "CTSB", "CLU", "CCL20", "CCL2", "BTC",
                                     "AREG"
                                  ],
              "senmayo":pd.read_excel("data/senescence_list.xlsx",sheet_name="SenMayo")["symbol"].tolist(),
              "cellage":pd.read_excel("data/senescence_list.xlsx",sheet_name="CellAge Senescence Genes")["Symbol"].tolist()
                }

test_genes["union"] = reduce(np.union1d, [test_genes["fridman"],
                    test_genes["sasp2"],
                    test_genes["senmayo"],
                    test_genes["cellage"]]
      )
test_genes["all_smoker"] = hlca_smoker.var_names.tolist()
test_genes["all_nonsmoker"] = hlca_nonsmoker.var_names.tolist()

## Modeling and gene selection

In [23]:
'''
Filter anndata by cell type, marker genes, and subjects. Subjects are selected by min_cells
Return filtered expression matrix and ages.
'''
def filter_anndata_single_ct(anndata, ct_column, ct, donor_column, age_column, marker_genes = None, min_cells = 20):
        
    # Keep rows annotated with current cell type, and columns annotated with marker genes,  
    ct_anndata = anndata[anndata.obs[ct_column] == ct, :]
    ct_anndata = ct_anndata[:, ct_anndata.var_names.isin(marker_genes)]
    
    # Select subjects having number of cells greater than min_cells 
    subjects = ct_anndata.obs[donor_column]
    subjects_count = subjects.groupby(subjects.values).count()
    selected_subjects = subjects_count.loc[subjects_count >= min_cells].index 
    
    # Further subset anndata using the selected subjects
    ct_anndata = ct_anndata[ct_anndata.obs[donor_column].isin(selected_subjects),]
    
    # Generate filtered expression matrix and ages
    expr = ct_anndata.to_df()
    expr.index = ct_anndata.obs[donor_column].values
    ages = ct_anndata.obs[age_column]
    ages.index = expr.index
        
    return expr, ages

Extract cell types with R2 >= 0.05

In [ ]:
import warnings
warnings.filterwarnings("ignore")

iterator = []
    
for idx,row in tqdm(table_smoker.iterrows(),total = table_smoker.shape[0]):
    if row["all_donor_num"] >= 10:
        expr, ages = filter_anndata_single_ct(hlca_smoker,
                                     ct_column = row["ann_level"],
                                     ct = row["cell_type"],
                                     donor_column = "donor_id",
                                     age_column = "age",
                                     marker_genes = test_genes[row["gene_type"]],
                                     min_cells = 20
                                    )
        iterator.append([
                            expr,
                            ages,
                            row["group"],
                            row["cell_type"],
                            row["ann_level"],
                            row["gene_type"],
                            row["use_pca"],
                            row["all_donor_num"]
                        ]
                       )

for idx,row in tqdm(table_nonsmoker.iterrows(),total = table_nonsmoker.shape[0]):
    if row["all_donor_num"] >= 10:
        expr, ages = filter_anndata_single_ct(hlca_nonsmoker,
                                     ct_column = row["ann_level"],
                                     ct = row["cell_type"],
                                     donor_column = "donor_id",
                                     age_column = "age",
                                     marker_genes = test_genes[row["gene_type"]],
                                     min_cells = 20
                                    )
        iterator.append([
                            expr,
                            ages,
                            row["group"],
                            row["cell_type"],
                            row["ann_level"],
                            row["gene_type"],
                            row["use_pca"],
                            row["all_donor_num"],
                        ]
                       )

In [25]:
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler

from functools import partial,reduce
from hyperopt import hp, Trials, fmin, tpe
from hyperopt import space_eval
import shap

Train and test models

In [26]:
def compute_features(expr, ages, mean_degree, var_degree, n_components, use_pca, pca_model):

    if use_pca and pca_model is None:
        pca_model = PCA(n_components=n_components, whiten=True)
        expr = pd.DataFrame(pca_model.fit_transform(expr), index = expr.index)
        expr.columns = [f"PC{i+1}" for i in range(expr.shape[1])]
    elif use_pca and pca_model is not None:
        expr = pd.DataFrame(pca_model.transform(expr), index = expr.index)
        expr.columns = [f"PC{i+1}" for i in range(expr.shape[1])]
        
    # Get each subject/individual's mean expression and polynomials of mean expressions.
    subjects = expr.index.to_list()
    expr_mean = expr.groupby(subjects).mean()
    expr_mean_poly = np.hstack([expr_mean**i for i in range(1, mean_degree+1)])
    feature_names = np.hstack([[f"{col}_mean^{deg}" for col in expr.columns] for deg in range(1,mean_degree+1)]) # Name the polynomial features
    expr_mean_poly = pd.DataFrame(
                        expr_mean_poly,
                        index = expr_mean.index,
                        columns = feature_names 
                    )
    
    if var_degree > 0:

        # Get each subject/indivisual's variances.
        expr_var = expr.groupby(subjects).var()
        expr_var.fillna(value=0,inplace=True) # This should not happen 
        expr_var_poly = np.hstack([expr_var**i for i in range(1, var_degree+1)])
        feature_names = np.hstack([[f"{col}_var^{deg}" for col in expr.columns] for deg in range(1,var_degree+1)]) # Name the polynomial features
        expr_var_poly = pd.DataFrame(
                        expr_var_poly,
                        index = expr_var.index,
                        columns = feature_names 
                    )

        # Concatenate mean and var polynomial features
        X = pd.concat([expr_mean_poly,expr_var_poly],axis = 1)
    else:
        X = expr_mean_poly
    
    # Get Y
    Y = ages.groupby(subjects).mean()
    
    return X,Y,pca_model

In [27]:
def tune(param, expr, ages):
    
    '''
    # Get Hyperparameters
    '''
    n_components = param['n_components']
    mean_degree = param['mean_degree']
    var_degree = param['var_degree']
    alpha = param['alpha']
    l1_ratio = param['l1_ratio']
    
    X = expr.copy()
    Y = ages.copy()
    
    # Center and scale the data
    X = StandardScaler().fit_transform(X)
    Y = Y.values
    Y = StandardScaler().fit_transform(Y.reshape(-1,1)).ravel()
    
    # Model fitting
    polyreg = make_pipeline(ElasticNet(max_iter=40000, alpha=alpha, l1_ratio=l1_ratio))
    polyreg.fit(X, Y)
        
    # Get predicted age for training data
    pred = polyreg.predict(X)
    
    # Return negative R2 as loss
    return(-r2_score(Y, pred))

In [28]:
def train_and_select(expr, ages, use_pca, param_space, n_hyper_eval, subject_type, ct):
    
    subjects = expr.index.unique()
    gene_names = expr.columns.tolist()
    
    X, Y, pca_model = compute_features(expr, ages, mean_degree=2, var_degree=2, n_components=10, use_pca = use_pca, pca_model=None)
    feature_names = X.columns.tolist()
    
    # Partial will freeze some arguments for tune()
    fmin_objective = partial(
                            tune,
                            expr=X,
                            ages=Y
                        )

    # Search for the best hyperparameters on training data
    param_best = fmin(fmin_objective,
                        space = param_space,
                        algo=tpe.suggest,
                        max_evals=n_hyper_eval,
                        verbose = False
                     )
    param_best = space_eval(param_space, param_best)

    X = StandardScaler().fit_transform(X)

    # Center and scale Y
    Y = Y.values
    Y = StandardScaler().fit_transform(Y.reshape(-1,1)).ravel()

    # Use the best hyperparameters to fit a model on training data
    polyreg = make_pipeline(ElasticNet(max_iter=40000, alpha=param_best['alpha'], l1_ratio=param_best['l1_ratio']))
    polyreg.fit(X, Y)

    # Extract non-zero feature indices
    nz_idx = np.where(polyreg[0].coef_ != 0)
    
    return X,Y,nz_idx,gene_names,feature_names,pca_model,polyreg

This function runs shap computation and gets the ranking score for each gene

In [29]:
def get_gene_score(X,Y,gene_names,feature_names,model,pca_model):
    
    # Get shap values with zeros as background
    background = np.zeros((1,X.shape[1]))
    e = shap.explainers.Linear(model, background)
    shap_values = e.shap_values(X)
    scores = np.abs(shap_values).sum(axis = 0)
    
    if pca_model is None:
        scores = scores.reshape(int(scores.shape[0]/len(gene_names)),len(gene_names))
        scores = scores.sum(axis = 0)
        scores = pd.DataFrame(scores,index = gene_names,columns = ["score"]).sort_values(by = "score", ascending = False)
    else:
        scores = scores.reshape(int(scores.shape[0]/pca_model.components_.shape[0]),pca_model.components_.shape[0])
        scores = scores.sum(axis = 0)
        eigen_mat = pca_model.components_
        eigen_mat_inv = np.linalg.pinv(eigen_mat)
        scores = np.abs(np.matmul(eigen_mat_inv, scores))
        scores = pd.DataFrame(scores,index = gene_names,columns = ["score"]).sort_values(by = "score", ascending = False)
    
    return scores

### Main analysis pipeline
Adjust n_jobs based on number of cores and memory available in your machine

In [28]:
from joblib import delayed, Parallel
import warnings

def run_all(expr, ages, group, ct, ct_column, gene_type, use_pca, donor_num_all, n_hyper_eval):
    warnings.filterwarnings("ignore")

    # Define hyperparameter search space
    param_space = {'n_components' : hp.choice('n_components', [10]),
                 'mean_degree': hp.choice('mean_degree', [2]),
                 'var_degree': hp.choice('var_degree', [2]), 
                 'alpha': hp.choice('alpha', [0.001, 0.01, 0.1, 1, 10, 100]), 
                 'l1_ratio': hp.uniform('l1_ratio', 0.1, 1.0)
        }
    
    # Model training and feature selection.
    X,Y,nz_idx,gene_names,feature_names,pca_model,polyreg = train_and_select(expr, ages, use_pca, param_space, n_hyper_eval = n_hyper_eval, subject_type = group, ct = ct)
    scores = get_gene_score(X,Y,gene_names,feature_names,polyreg[0],pca_model)
    print(f"==========Finished==========\n Ann level:{ct_column}\n Group: {group}\nCell type: {ct}\nGene type: {gene_type}\nUse pca:{use_pca}")
    
    return ct_column,group,ct,gene_type,use_pca,nz_idx,gene_names,feature_names,pca_model, donor_num_all, scores

res = Parallel(n_jobs = 100)(delayed(run_all)(
                                          expr,
                                          ages,
                                          group,
                                          ct,
                                          ct_column,
                                          gene_type,
                                          use_pca,
                                          donor_num_all,
                                          n_hyper_eval = 30
                                            ) for expr,ages,group,ct,ct_column,gene_type,use_pca,donor_num_all in tqdm(iterator))

100%|██████████| 900/900 [00:26<00:00, 34.23it/s]


Save results

In [32]:
import pickle
import os
if not os.path.isdir("results/gene_selection_ranking"):
    os.mkdir("results/gene_selection_ranking")    
    
with open('/results/gene_selection_ranking/all_selected_hlca_nonsmoker.pickle', 'wb') as handle:
    pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)